<a href="https://colab.research.google.com/github/Disha-Sikka/FineTune_and_GradCam/blob/main/FineTune_%26_GradCam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!kaggle datasets download -d puneet6060/intel-image-classification
!unzip -q intel-image-classification.zip


Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
 96% 334M/346M [00:05<00:00, 42.8MB/s]
100% 346M/346M [00:05<00:00, 62.6MB/s]


In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform= transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [3]:
train_dataset= datasets.ImageFolder('seg_train/seg_train', transform=transform)
test_dataset= datasets.ImageFolder('seg_test/seg_test', transform=transform)

train_loader= DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader= DataLoader(test_dataset, batch_size=32)


In [4]:
import torch
import torch.nn as nn
import torchvision.models as models

In [5]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= models.resnet18(pretrained= True)
num_ftrs= model.fc.in_features
model.fc= nn.Linear(num_ftrs,6)
model= model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
import torch.optim as optim
import os
from tqdm import tqdm

In [15]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(), lr=0.001)
os.makedirs('checkpoints', exist_ok=True)

for epochs in range(10):
  model.train()
  running_loss=0
  correct=0
  total=0

  for images, labels in tqdm(train_loader, desc=f"Epoch {epochs+1}/{10}"):
    images, labels= images.to(device), labels.to(device)
    outputs= model(images)
    loss= criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss+= loss.item()
    _, predicted= outputs.max(1)
    total += labels.size(0)
    correct+= (predicted==labels).sum().item()

  train_loss= running_loss/ len(train_loader)
  train_acc= 100* correct/total
  print(f"Epoch [{epochs+1}/10], Accuracy: {train_acc:.2f}, Loss: {train_loss:.4f}")
  checkpoint_path = f"checkpoints/resnet18_epoch{epochs+1}.pt"
  torch.save({
      'epoch': epochs+1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'train_loss': train_loss,
      'train_acc': train_acc,
  }, checkpoint_path)
  print(f"✅ Checkpoint saved: {checkpoint_path}")



Epoch 1/10: 100%|██████████| 439/439 [01:05<00:00,  6.75it/s]


Epoch [1/10], Accuracy: 92.51, Loss: 0.2125
✅ Checkpoint saved: checkpoints/resnet18_epoch1.pt


Epoch 2/10: 100%|██████████| 439/439 [01:05<00:00,  6.74it/s]


Epoch [2/10], Accuracy: 94.31, Loss: 0.1677
✅ Checkpoint saved: checkpoints/resnet18_epoch2.pt


Epoch 3/10: 100%|██████████| 439/439 [01:06<00:00,  6.61it/s]


...

Epoch [8/10], Accuracy: 98.25, Loss: 0.0537
✅ Checkpoint saved: checkpoints/resnet18_epoch8.pt


Epoch 9/10: 100%|██████████| 439/439 [01:06<00:00,  6.61it/s]


Epoch [9/10], Accuracy: 98.04, Loss: 0.0602
✅ Checkpoint saved: checkpoints/resnet18_epoch9.pt


Epoch 10/10: 100%|██████████| 439/439 [01:06<00:00,  6.58it/s]


Epoch [10/10], Accuracy: 98.56, Loss: 0.0448
✅ Checkpoint saved: checkpoints/resnet18_epoch10.pt
